## Install required libraries for solution 

In [ ]:
!pip install pickle5

In [ ]:
!pip install cloudpickle==2.1.0

In [ ]:
!pip install gym==0.21.0

In [ ]:
!pip install gym-retro

In [ ]:
!pip install opencv-contrib-python --user

In [ ]:
!pip install matplotlib

In [ ]:
!pip install py-trees

In [ ]:
!pip install numpy

In [ ]:
!pip install tensorflow

## Import All Required Libraries

In [1]:
# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
# Import matplotlib for plotting the image
from matplotlib import pyplot as plt
import retro
# Import time to slow down game
import time

import cv2 as cv

#To be able to log custom scalars
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import gym
#To be ablle to create Behavior tree
import py_trees.decorators
import py_trees.display
import random
import py_trees.console as console

# import the modules
import os
from os import listdir

# used for creating date in required formate
from datetime import date


## Setup paths to log , record agent game play

In [2]:
LOG_DIR = './logs_final/final_results_pre_programmed'
RECORD_PATH= './RecordAgentsGamePlay/final_results_pre_programmed'
model_name = 'final_results_pre_programmed_' + date.today().strftime('%Y-%m-%d')
character_imgs_path = "imgs/character"
enemies_imgs_path = "imgs/enemies" 

## Helper class Vision used for object detection in game frame

In [3]:
#Adapted from https://github.com/learncodebygaming/opencv_tutorials/blob/master/006_hsv_thresholding/vision.py
# removed elements that where not needed
#Change made removed constant TRACKBAR_WINDOW as do not require it 
#Change made removed init_control_gui
#Change made removed get_hsv_filter_from_controls
#Change made removed shift_channel
#Change made removed apply_hsv_filter
#Change made added __iter__ function to be able loop through list of Vision objects
class Vision:

    # properties
    needle_img = None
    needle_w = 0
    needle_h = 0
    method = None

    # constructor
    def __init__(self, needle_img_path, method=cv.TM_CCOEFF_NORMED):
        # load the image we're trying to match
        # https://docs.opencv.org/4.2.0/d4/da8/group__imgcodecs.html
        self.needle_img = cv.imread(needle_img_path, 0)

        # Save the dimensions of the needle image
        self.needle_w = self.needle_img.shape[1]
        self.needle_h = self.needle_img.shape[0]

        # There are 6 methods to choose from:
        # TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED
        self.method = method
        

    def find(self, haystack_img, threshold=0.5, max_results=10):
        # run the OpenCV algorithm
        result = cv.matchTemplate(haystack_img, self.needle_img, self.method)

        # Get the all the positions from the match result that exceed our threshold
        locations = np.where(result >= threshold)
        locations = list(zip(*locations[::-1]))

        # if we found no results, return now. this reshape of the empty array allows us to 
        # concatenate together results without causing an error
        if not locations:
            return np.array([], dtype=np.int32).reshape(0, 4)

        # You'll notice a lot of overlapping rectangles get drawn. We can eliminate those redundant
        # locations by using groupRectangles().
        # First we need to create the list of [x, y, w, h] rectangles
        rectangles = []
        for loc in locations:
            rect = [int(loc[0]), int(loc[1]), self.needle_w, self.needle_h]
            # Add every box to the list twice in order to retain single (non-overlapping) boxes
            rectangles.append(rect)
            rectangles.append(rect)
        # Apply group rectangles.
        # The groupThreshold parameter should usually be 1. If you put it at 0 then no grouping is
        # done. If you put it at 2 then an object needs at least 3 overlapping rectangles to appear
        # in the result. I've set eps to 0.5, which is:
        # "Relative difference between sides of the rectangles to merge them into a group."
        rectangles, weights = cv.groupRectangles(rectangles, groupThreshold=1, eps=0.5)

        # for performance reasons, return a limited number of results.
        # these aren't necessarily the best results.
        if len(rectangles) > max_results:
            print('Warning: too many results, raise the threshold.')
            rectangles = rectangles[:max_results]

        return rectangles

    # given a list of [x, y, w, h] rectangles returned by find(), convert those into a list of
    # [x, y] positions in the center of those rectangles where we can click on those found items
    def get_click_points(self, rectangles):
        points = []

        # Loop over all the rectangles
        for (x, y, w, h) in rectangles:
            # Determine the center position
            center_x = x + int(w/2)
            center_y = y + int(h/2)
            # Save the points
            points.append((center_x, center_y))

        return points

    # given a list of [x, y, w, h] rectangles and a canvas image to draw on, return an image with
    # all of those rectangles drawn
    def draw_rectangles(self, haystack_img, rectangles):
        # these colors are actually BGR
        line_color = (0, 255, 0)
        line_type = cv.LINE_4

        for (x, y, w, h) in rectangles:
            # determine the box positions
            top_left = (x, y)
            bottom_right = (x + w, y + h)
            # draw the box
            cv.rectangle(haystack_img, top_left, bottom_right, line_color, lineType=line_type)

        return haystack_img

    # given a list of [x, y] positions and a canvas image to draw on, return an image with all
    # of those click points drawn on as crosshairs
    def draw_crosshairs(self, haystack_img, points):
        # these colors are actually BGR
        marker_color = (255, 0, 255)
        marker_type = cv.MARKER_CROSS

        for (center_x, center_y) in points:
            # draw the center point
            cv.drawMarker(haystack_img, (center_x, center_y), marker_color, marker_type)

        return haystack_img


    # I added this change to be be able iterate over obect as need to loop over list Vision objects
    def __iter__(self):
        return self



## Helper functions used for object detection in game frame 

In [4]:
#Function take one augment an folder path which should contain images your trying to find in the game frame. 
#Loops over each image in the folder proceeds to calls Vision with image as arg the created Vision obj is appended to a list. 
#The function returns a list of Vision objects. Used to find location of the enemies and game character in game.
def get_match_template_images(folder_dir):
    
    # create a empty list to hold Vision objects
    img_list = []

    #loop over each img file to read it and add it to list
    for image in os.listdir(folder_dir):
 
        # check if the image ends with png or jpg or jpeg
        if (image.endswith(".png") or image.endswith(".jpg") or image.endswith(".jpeg")):
            img_list.append(Vision(folder_dir+ '/'+ image))
        
    return img_list

In [5]:
#Function built to return a list of rectangle locations from a list of vision objects (of enemies or game character). 
#The inputs of the function are current game frame ,list Vision objects from calling get_match_template_images, 
#threshold for the match template should use and max number of results.
def get_object_detection_rects(haystack_img,match_template_images,threshold=0.85, max_results=10):
    
    # create a array to hold the rectangle coordinates with required shape
    rectangles = np.array([], dtype=np.int32).reshape(0, 4)
    
    # loop over each Vision object that you would like to match template for aka object detection 
    for match_image in match_template_images:
         
        # create a empty array with correct shape     
        temp = np.array([], dtype=np.int32).reshape(0, 4)
        # do object detection using match template
        temp = match_image.find(cv2.cvtColor(haystack_img, cv.COLOR_BGR2GRAY ), threshold,max_results)
        
        #Is there objects found in current game frame
        if len(temp) > 0:
            #combine rectangles (need to stack them in shape we require of (0,4) )
            rectangles =  np.vstack((rectangles, temp))
            
    # To avoid overlapping rectangles we need to group them with certain threshold only if more than one rectangle in list 
    if len(rectangles) > 1:
        # group rectangles
        rectangles, weights = cv.groupRectangles(rectangles, groupThreshold=1, eps=0.5)    
    
    return rectangles

In [6]:
#Function to deteremine where there are more enemies either the right or left will favourer the right, that are
#within range of character position if character position is known
def direction_to_go_when_enemies_on_screen(character_pos,enemies_center_points):
    
    # If there no enemies then go right as one of goals of game it to get to the right where boss is 
    if len(enemies_center_points) < 1:
        return 1
    
    # create some variables to hold count 
    grt = 0
    ls = 0

    # loop over each enemy to determine if they are within firing range
    for point in enemies_center_points :

        # calulate the distance using b/w a and b
        distance = ((character_pos[0] - point[0])**2 + (character_pos[1] - point[1])**2)**0.5
        
        # Is enemy within range ?
        if int(distance) < 5:
    
            # Decide if the enemy is on right ?
            if point[0] >= character_x :
                grt += 1
            else:
                ls += 1
        
    # Are there more enemies on the right ?
    if grt >= ls :
        return 1
    else :
        return -1


In [7]:
#This function takes a list which has a x and y position in it.Returns avg position of character
#as there could possible return more than one location for character position or no locations.
#I summed x and y positon of the character so character postion can be passed to Behaviour nodes called ShouldCharacterAttackRight.
def get_avg_pos_for_character(character_center_points):
    
    # No characters found so retrun default location
    if len(character_center_points) < 1:
        return [0,0]
    # One match for character pos can just retrun it back
    elif len(character_center_points) == 1 :
        return character_center_points[0]
    else:
        # need to sum up the x and y positions then divided by number positions there are
        x_points_summed = sum(x for x, y in character_center_points)
        x_points_summed // len(character_center_points)
        
        y_points_summed = sum(y for x, y in character_center_points)
        y_points_summed // len(character_center_points)
        
        return [x_points_summed, y_points_summed]
    

# Create game environment wrapper

In [8]:
#Adapted class called StreetFighter from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
#Made changes to the __init__ function to allow the user the option to record the game play , 
#changed it work for Alien Soldier enviroment ,
#added logic to set buttons and game state name , created custom variables character_match_images,character_center_points
#character_center_points ,enemies_match_images and enemies_center_points.
#Made changes to step function added logic to find enemies and game character on current game frame to then updated required 
# varaibles with location of found objects.

class AlienSoldier(Env): 
    def __init__(self,character_imgs_path, enemies_imgs_path,record_results=False,record_path=''):
        super().__init__()
        
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Set the buttons as they needed when wrapped with AlienSoldierDiscretizer
        self.buttons = ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
                
        # Startup and instance of the game 
        if record_results:
            # Startup and instance of the game 
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED, record=record_path)
        else:
            # Startup and instance of the game 
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        
        #set the game state name so able to access which level the game is on
        self.statename = self.game.statename
        
        #set up the needle images for character using get_match_template_images which uses Vision classs
        self.character_match_images = get_match_template_images(character_imgs_path)
        #set up array to hold character rects 
        self.character_rectangles = np.array([], dtype=np.int32).reshape(0, 4)
        #set up array to hold character x and y coordinates 
        self.character_center_points = np.array([], dtype=np.int32).reshape(0, 2)
        
        #set up the needle images for character using get_match_template_images which uses Vision classs
        self.enemies_match_images = get_match_template_images(enemies_imgs_path)
        #set up array to hold enemies rects 
        self.enemies_rectangles = np.array([], dtype=np.int32).reshape(0, 4)
        #set up array to hold enemies x and y coordinates 
        self.enemies_center_points = np.array([], dtype=np.int32).reshape(0, 2)
        
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)

        # object detection using match template find game character and enemies rects
        self.character_rectangles = get_object_detection_rects(obs,self.character_match_images,threshold=0.85, max_results=10)
        self.enemies_rectangles = get_object_detection_rects(obs,self.enemies_match_images,threshold=0.85, max_results=10)
        # need the rects turned into x and y coordinate to be used by Behavior tree
        self.enemies_center_points = self.character_match_images[0].get_click_points(self.enemies_rectangles)
        self.character_center_points = self.character_match_images[0].get_click_points(self.character_rectangles)
        
        # Would like to draw found rects on frame but need to place to hold grouped rects so there no overlapping
        grouped_rectangles = np.array([], dtype=np.int32).reshape(0, 4)
        
        #combine rectangles if there rects in both enemies and game character
        if len(self.character_rectangles) > 0 and len(self.enemies_rectangles) > 0:
            grouped_rectangles =  np.vstack((self.character_rectangles, self.enemies_rectangles))
        elif len(self.character_rectangles) > 0 :
            grouped_rectangles = self.character_rectangles
        elif len(self.enemies_rectangles) > 0 :
            grouped_rectangles = self.enemies_rectangles
        
        # Draw rects onto current game frame , using the first vision object does not matter which one is 
        #used just want to be able to call function draw_rectangles
        output_image = self.character_match_images[0].draw_rectangles(cv2.cvtColor(obs, cv.COLOR_BGR2GRAY ), grouped_rectangles)

        # display the processed image
        cv.imshow('Match Template', output_image)
        
        obs = self.preprocess(obs) 
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    
    def render(self, *args, **kwargs):
        self.game.render()
        
        
    def close(self):
        self.game.close()

## Build Wrapper for Gym Env So Actions A Just Combo's Needed

In [9]:
#Code not mine taken from https://github.com/openai/retro/blob/master/retro/examples/discretizer.py 
#Made change, update class to AlienSoldierDiscretizer instead of SonicDiscretizer , also changed the combo list added in the class to 
#be one that could be used in Alien Soldier
class Discretizer(gym.ActionWrapper):
    """
    Wrap a gym environment and make it use discrete actions.
    Args:
        combos: ordered list of lists of valid button combinations
    """

    def __init__(self, env, combos):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.MultiBinary)
        buttons = env.unwrapped.buttons
        self._decode_discrete_action = []
        for combo in combos:
            arr = np.array([False] * env.action_space.n)
            for button in combo:
                arr[buttons.index(button)] = True
            self._decode_discrete_action.append(arr)

        self.action_space = gym.spaces.Discrete(len(self._decode_discrete_action))

    def action(self, act):
        return self._decode_discrete_action[act].copy()


class AlienSoldierDiscretizer(Discretizer):
    """
    based on https://github.com/openai/retro-baselines/blob/master/agents/sonic_util.py
    """
    
    def __init__(self, env):
        # Added own combo list
        super().__init__(env=env, combos=[['LEFT'], 
                                          ['RIGHT'],
                                          ['C'],
                                          ['C','LEFT'],
                                          ['C','RIGHT'],
                                          ['B'],
                                          ['B','B'],
                                          ['LEFT','B','B'],
                                          ['RIGHT','B','B']
                                         ])

# Create Custom Behaviours For Behaviour Tree

In [10]:
# Based PreformAction class from https://py-trees.readthedocs.io/en/devel/behaviours.html
# Built a custom node for behaviour tree to set the action to perform in the black blackboard variable called action.
# The agent will read the black blackboard variable called action then procceed to pass that action to environment step method 
# perform that action.
class PreformAction(py_trees.behaviour.Behaviour):
    """Updates the blackboard variable called action to set the action the agent should perform next when it reads the action.
    """

    def __init__(self, name ,action,action_name,blackboard_name):
        """Configure init of behaviour node"""
        
        super(PreformAction, self).__init__(name)
        
        #Set variable to hold which action needs to be set to blackboard action
        self.action = action
        #Set variable to hold human friendly name for loggin purposes 
        self.action_name = action_name
        
        # attach the blackboard to variable in the init function to be able to read and write to shared blackboard 
        self.blackboard = self.attach_blackboard_client(name=blackboard_name,namespace="parameters")
        
        # To be able to read and write to a variables in the blackboard you need register it with access level and variable 
        # need to access.
        self.blackboard.register_key(key="action", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="action", access=py_trees.common.Access.WRITE)
        
    def setup(self):
        """No delayed initialisation."""

    def initialise(self):
        """Reset"""
                

    def update(self):
        """Update."""
        
        # Used to determine if the node was run successfully 
        decision = random.choice([True, True])
        
        # When action is go right update the decision to have possibly of failing so allows chance for action jump to happen
        # When action is attacking update the decision to have possibly of failing as character does not procceed forward if only
        # attacking in one place
        if self.action == 1 or self.action == 7 or self.action == 8:
            decision = random.choice([True, False,True,True,False,True,True])
            
        #Check if true then node ran successfully then update the blackboard variable action
        if decision:
            self.feedback_message = "action perform " + self.action_name
            self.logger.debug("%s" % self.action_name)
            self.blackboard.action = self.action 
            return py_trees.common.Status.SUCCESS
        else:
            # node fails move onto next node
            self.feedback_message = "failed to perform " + self.action_name
            return py_trees.common.Status.FAILURE
        
        
    def terminate(self, new_status):
        """Nothing to clean up."""
        


In [11]:
# Based TakenDamage class from https://py-trees.readthedocs.io/en/devel/behaviours.html
# Built a custom node for behaviour tree to indicate if agent should attack or not
class TakenDamage(py_trees.behaviour.Behaviour):
    """
    A node to determine if the game character is under attack or not 
    """

    def __init__(self, name , blackboard_name):
        """Configure init of behaviour node"""
        
        super(TakenDamage, self).__init__(name)
        
        # attach the blackboard to variable in the init function to be able to read and write to shared blackboard
        self.blackboard = self.attach_blackboard_client(name=blackboard_name,namespace="parameters")
        
        # To be able to read and write to a variables in the blackboard you need register it with access level and variable 
        # need to access.
        self.blackboard.register_key(key="IsUnderattack", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="IsUnderattack", access=py_trees.common.Access.WRITE)
        self.blackboard.register_key(key="amount_damage_taken", access=py_trees.common.Access.READ)

        
    def setup(self):
        """No delayed initialisation"""
        
    def initialise(self):
        """Reset"""
                

    def update(self):
        """Update."""
        
        self.logger.debug("%s.update()" % (self.__class__.__name__))
        self.logger.debug("amount_damage_taken is %s " % str(self.blackboard.amount_damage_taken))
        
        # Read from blackboard variable  amount_damage_taken to determine if damage was taken        
        if int(self.blackboard.amount_damage_taken) > 0 :
            #Update the blackboard varaible IsUnderattack to true to indicate the game character is under attack 
            # The node will be set to succcess 
            self.feedback_message = "damage taken : " + str(self.blackboard.amount_damage_taken)
            self.blackboard.IsUnderattack = True
            return py_trees.common.Status.SUCCESS 
            
        else:
            # As not under attack set to success as false 
            self.blackboard.IsUnderattack = False
            self.feedback_message = "no damage taken"
            return py_trees.common.Status.FAILURE
        
        
    def terminate(self, new_status):
        """Nothing to clean up"""
        


In [12]:
# Based IsThereEnemies class from https://py-trees.readthedocs.io/en/devel/behaviours.html
# Built a custom node for behaviour tree to deteremine if there are enemies to attack.There is logic in place to decide of 
# if the enemies are within range if game character position if known, there no point firing if enemies out of firing range.
class IsThereEnemies(py_trees.behaviour.Behaviour):
    """.
    Deciding if there are enemies within current game frame. Added conditional statements of if character position is known
    the decide if enemies are with range if true then blackboard variable enemies is set to true else false. If the game 
    characters position is not known if there any enemies on screen then set then blackboard variable enemies is set 
    to true else false.
    """

    def __init__(self, name , blackboard_name):
        """Configure init of behaviour node"""
        
        super(IsThereEnemies, self).__init__(name)
        
        # attach the blackboard to variable in the init function to be able to read and write to shared blackboard
        self.blackboard = self.attach_blackboard_client(name=blackboard_name,namespace="parameters")
        
        # To be able to read and write to a variables in the blackboard you need register it with access level and variable 
        # need to access.
        self.blackboard.register_key(key="character_center_points", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="enemies_center_points", access=py_trees.common.Access.READ)

        self.blackboard.register_key(key="enemies", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="enemies", access=py_trees.common.Access.WRITE)
        
    def setup(self):
        """No delayed initialisation"""
        
    def initialise(self):
        """Reset"""
        
    def update(self):
        """Update."""
        
        # Messages to help with logging
        self.logger.debug("%s.update()" % (self.__class__.__name__))
        self.logger.debug("enemies_center_points : %s " % str(self.blackboard.enemies_center_points))
        
        # Is the game character position known ?
        if len(self.blackboard.character_center_points) > 0 :
            
            # Set variable which holds if enemy is with range and default it to false
            enemies_with_in_range = False
            #Get average position of game character encase there more than one match  
            character_pos = get_avg_pos_for_character(self.blackboard.character_center_points)
            
            # Loop over each enemy 
            for point in self.blackboard.enemies_center_points :
                
                # Get distance from game character to the enemy using  b/w a and b
                distance = ((character_pos[0] - point[0])**2 + (character_pos[1] - point[1])**2)**0.5
                
                # Is enemy within range ?
                if int(distance) < 5:
                    
                    enemies_with_in_range = True   
                    
            # Is enemy within range ?
            if enemies_with_in_range:
                self.feedback_message = "Enemies in range: Yes" 
                self.blackboard.enemies = True
                # Set node to success 
                return py_trees.common.Status.SUCCESS
            else:
                self.feedback_message = "Enemies in range : No"
                self.blackboard.enemies = False
                # Set node to failure
                return py_trees.common.Status.FAILURE
        # Are there nay enemies in the game frame ?
        elif len(self.blackboard.enemies_center_points) > 0:
            self.feedback_message = "Enemies : Yes" 
            self.blackboard.enemies = True
            # Set node to success 
            return py_trees.common.Status.SUCCESS
        else:
            # No enemies in game frame
            self.feedback_message = "Enemies : No"
            self.blackboard.enemies = False
            # Set node to failure
            return py_trees.common.Status.FAILURE
        
    def terminate(self, new_status):
        """Nothing to clean up"""

In [13]:
# Based ShouldCharacterAttackRight class from https://py-trees.readthedocs.io/en/devel/behaviours.html
# Built a custom node for behaviour tree 
class ShouldCharacterAttackRight(py_trees.behaviour.Behaviour):
    """
    """

    def __init__(self, name , blackboard_name):
        """Configure init of behaviour node"""
        
        super(ShouldCharacterAttackRight, self).__init__(name)
        
        # attach the blackboard to variable in the init function to be able to read and write to shared blackboard
        self.blackboard = self.attach_blackboard_client(name=blackboard_name,namespace="parameters")
        # To be able to read and write to a variables in the blackboard you need register it with access level and variable 
        # need to access.
        self.blackboard.register_key(key="character_center_points", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="enemies_center_points", access=py_trees.common.Access.READ)

    def setup(self):
        """No delayed initialisation"""
        
    def initialise(self):
        """Reset"""
        
    def update(self):
        """Update."""  
        
        #Get average position of game character encase there is more than one match
        character_pos = get_avg_pos_for_character(self.blackboard.character_center_points)
        
        # Is the game character position Known ? Default value is [0,0]
        if character_pos == [0,0]:
            self.feedback_message = "go right because unable to find character x pos "
            # Set node to success 
            return py_trees.common.Status.SUCCESS
        
        # Decide which direction the game character should go by calling custom function which checks from
        # game position if there more enemies are right or left 
        direction = direction_to_go_when_enemies_on_screen(character_pos,self.blackboard.enemies_center_points)
        
        # Is direction go right ?
        if direction == 1:
            self.feedback_message = "go right as there more enemies on right "
            # Set node to success 
            return py_trees.common.Status.SUCCESS
        else:
            # Go left
            self.feedback_message = "should go left as there more enemies on left"
            # Set node to failure 
            return py_trees.common.Status.FAILURE
        
        
    def terminate(self, new_status):
        """Nothing to clean up"""

## BlackBoard helper function

In [14]:
# A function to create the blackboard and register the variables against it
def createBlackBoard(name, namespace):
    
    # Create blackboard
    blackboard = py_trees.blackboard.Client(name=name,namespace=namespace)

    #Setup write access
    blackboard.register_key(key="action", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="health", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="previous_health", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="amount_damage_taken", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="score", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="previous_score", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="time", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="IsUnderattack", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="character_center_points", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="enemies_center_points", access=py_trees.common.Access.WRITE)
    blackboard.register_key(key="enemies", access=py_trees.common.Access.WRITE)
    
    #Setup read access
    blackboard.register_key(key="action", access=py_trees.common.Access.READ)
    blackboard.register_key(key="health", access=py_trees.common.Access.READ)
    blackboard.register_key(key="previous_health", access=py_trees.common.Access.READ)
    blackboard.register_key(key="amount_damage_taken", access=py_trees.common.Access.READ)
    blackboard.register_key(key="score", access=py_trees.common.Access.READ)
    blackboard.register_key(key="previous_score", access=py_trees.common.Access.READ)
    blackboard.register_key(key="time", access=py_trees.common.Access.READ)
    blackboard.register_key(key="IsUnderattack", access=py_trees.common.Access.READ)
    blackboard.register_key(key="character_center_points", access=py_trees.common.Access.READ)
    blackboard.register_key(key="enemies_center_points", access=py_trees.common.Access.READ)
    blackboard.register_key(key="enemies", access=py_trees.common.Access.READ)

    # Return the created blackboard
    return blackboard



In [15]:
# A function to set blackboard variables to default values
def resetBlackBoardValues(blackboard):
    
    blackboard.action = 1
    blackboard.health = 512
    blackboard.previous_health = 512
    blackboard.score = 0
    blackboard.previous_score = 0
    blackboard.time = 157
    blackboard.amount_damage_taken = 0
    blackboard.IsUnderattack = False
    blackboard.character_center_points = np.array([], dtype=np.int32).reshape(0, 2)
    blackboard.enemies_center_points = np.array([], dtype=np.int32).reshape(0, 2)
    blackboard.enemies = False

## Setup Blackboard , Register Variables And initialize default values

In [16]:
# Call function to create blackboard
blackboard = createBlackBoard("Global", "parameters")
# Call function to initialize blackboard varaibles to default values 
resetBlackBoardValues(blackboard)

## Behaviour Tree  -> Create Nodes And Leaf Nodes Of The Tree

In [17]:
def createBehaviourTree(name_of_tree, print_tree):
    # Create root of the tree and pass in required name of treee
    root = py_trees.composites.Selector(name="Agent")

    # Child of Agent, Setup sequence to determine if agent should go right
    proceed_forward = py_trees.composites.Sequence("Go Forward")
    # Child of Go Forward , using a inverter node check calls custom node TakenDamage to retrun if 
    #damage being taken by game character this inverts the result for example if no damage is taken it retruns true so 
    # call PreformAction passing in action of go right
    not_under_attack = py_trees.decorators.Inverter(
        name="Not Under Attack",
        child=TakenDamage(name="Check IF Taken Damage", blackboard_name="Global")
    )
    # Child of Not Under Attack , performs action go right
    go_right = PreformAction(name="Move Right", action=1,action_name="move right",blackboard_name="Global")


    # Child of Agent, Setup a composite node to determine if agent should attack
    attack_emeny = py_trees.composites.Sequence(name="Attack Enemy")
    # Child of Attack Enemy , check if there enemies around
    enemies_on_screen = IsThereEnemies(name="Check IF There Are Enemies Around", blackboard_name="Global")
    #Child of Attack Enemy , Setup a composite node to decide which direction to attack
    direction_to_attack = py_trees.composites.Selector(name="Direction To Attack")
    #Child of Direction To Attack, Setup a selector node which goes through each node below until one returns success 
    are_there_more_enemies_on_right = py_trees.composites.Sequence(name="Are There More Enemies On The Right")
    #Child of Are There More Enemies On The Right, checks if there agent should attack right
    should_go_right_and_attack = ShouldCharacterAttackRight(name="Should Character Attack Right", blackboard_name="Global")
    #Child of Are There More Enemies On The Right, peforms action of attack right
    attack_right = PreformAction(name="Attack Right", action=8,action_name="Attack Right",blackboard_name="Global")
    #Child of Direction To Attack, performs action of attack keft 
    attack_left = PreformAction(name="Attack Left", action=7,action_name="Attack Left",blackboard_name="Global")

    #Child of Agent , Setup composite node
    need_to_jump = py_trees.composites.Selector(name="Path To Blocked")
    #Child of Path To Blocked , peforms the action of jump
    jump = PreformAction(name="Jump", action=2,action_name="jump",blackboard_name="Global")

    # Child of Agent, performs action of go left
    proceed_left = PreformAction(name="Go Left", action=0,action_name="move left",blackboard_name="Global")


    # Add childen to root of behaviour tree 
    root.add_children([attack_emeny,proceed_forward,need_to_jump,proceed_left])
    # Add child edto node proceed_forward
    proceed_forward.add_children([not_under_attack, go_right])
    # Add childed to attack_emeny
    attack_emeny.add_children([enemies_on_screen, direction_to_attack])
    # Add childed to direction_to_attack
    direction_to_attack.add_children([are_there_more_enemies_on_right,attack_left])
    # Add childen to are_there_more_enemies_on_right
    are_there_more_enemies_on_right.add_children([should_go_right_and_attack,attack_right])
    # Add child to need_to_jump
    need_to_jump.add_children([jump])

    # Create behaviour tree
    behaviour_tree = py_trees.trees.BehaviourTree(root=root)
    
    # Print the layout of the tree
    if print_tree : 
        print(py_trees.display.unicode_tree(root=root))
      
    # setup tree  
    behaviour_tree.setup(timeout=15)
    
    return behaviour_tree

## Helper function for agent

In [18]:
def print_tree(tree):
    pass
#     print(py_trees.display.unicode_tree(root=tree.root, show_status=True))
    
def shutdown(behaviour_tree):
    behaviour_tree.interrupt()

In [19]:
# A function to retrun the level from state name in the retro enviroment. The state name contains alot string data 
# around it e.g DefaultSettings.Level1.state only interested in the level int as would like to log which level agnet ends on
def getLevelFromStateName(state_name):
    
    # initializing substrings
    start_str = "evel"
    end_str = ".state"
 
    # getting index of substrings
    start_index = state_name.find(start_str)
    end_index = state_name.find(end_str)
    
    # sub string the level from state name as we only want level as number 
    level = state_name[start_index + len(start_str): end_index]

    # convert the level to int and return it 
    return int(level)

In [20]:
# Function to start game environment and have the pre-programmed agent play the number games passed to function  
def preprogrammedAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,RecordGamePlay,NumerOfGamesToPlay,
                               behaviour_tree,behaviour_tree_logging_level,blackboard):

    # Setup Behaviour Tree logging level
    if behaviour_tree_logging_level == 0:
        py_trees.logging.level = py_trees.logging.Level.DEBUG
    elif behaviour_tree_logging_level == 1:
        py_trees.logging.level = py_trees.logging.Level.INFO
    elif behaviour_tree_logging_level == 2:
        py_trees.logging.level = py_trees.logging.Level.WARN
    elif behaviour_tree_logging_level == 3:
        py_trees.logging.level = py_trees.logging.Level.ERROR

    # Create environment 
    env = AlienSoldier(character_imgs_path, enemies_imgs_path,RecordGamePlay,RECORD_PATH) 
    # Wrap environment with AlienSoldierDiscretizer so enviroment action space is dicrete and no longer MultiBinary
    env = AlienSoldierDiscretizer(env)
    
    # Setup writer which will log the scalar values from end of agent game play
    writer = tf.summary.create_file_writer(LOG_DIR,name=model_name)
    
    # Starts up the game environment
    print("------------------------------------")
    # Reset game to starting state
    obs = env.reset()
    # Set done flag to flase this indicates if agent game is done
    done = False
    
    # Loop over number games to play 
    for game in range(NumerOfGamesToPlay): 
        print("Game Num" ,game + 1)
    
        #Check if agents game has ended
        while not done: 
            
            # Use the Behaviour tree to determine what action to pass to step method in game enviroment
            # by updating the action varaible in blackboard
            behaviour_tree.tick_tock(
            period_ms=0,
            number_of_iterations=1,
            pre_tick_handler=None,
            post_tick_handler=print_tree
            )
            
            # Perform action
            obs, reward, done, info = env.step(blackboard.action)
            
            # Update blackboard varaibles used in the Behaviour tree
            blackboard.previous_health = blackboard.health
            blackboard.health =  info["health"]
            blackboard.previous_score = blackboard.score
            blackboard.score = info["score"]
            blackboard.time = info["time"]
            blackboard.amount_damage_taken = blackboard.previous_health -  blackboard.health 
            blackboard.character_center_points = env.character_center_points
            blackboard.enemies_center_points = env.enemies_center_points
            
            if blackboard.previous_health != blackboard.health:
                blackboard.IsUnderattack = True
            
            # Render the game frame 
            env.render()
            
            #Check if game is done, if true then proceed to log scalar values to writer to be logged 
            if done: 
                print("Game over")
                print("states",info)
                
                # Log the values to writer 
                with writer.as_default():
                    tf.summary.scalar("score", info["score"], step=game + 1)
                    tf.summary.scalar("time", info["time"], step=game + 1)
                    tf.summary.scalar("health", info["health"], step=game + 1)
                    tf.summary.scalar("level", getLevelFromStateName(env.statename), step=game + 1)
                    writer.flush()
                print("------------------------------------")
        # Reset obs so game can restart
        obs = env.reset()
        done = False
        resetBlackBoardValues(blackboard)
    # Close enviroment
    env.close()

# Start game enviroment and start agent playing game for required amount of games 

In [21]:
# Create Behaviour tree
behaviour_tree = createBehaviourTree("Agent", True)

[o] Agent
    {-} Attack Enemy
        --> Check IF There Are Enemies Around
        [o] Direction To Attack
            {-} Are There More Enemies On The Right
                --> Should Character Attack Right
                --> Attack Right
            --> Attack Left
    {-} Go Forward
        -^- Not Under Attack
            --> Check IF Taken Damage
        --> Move Right
    [o] Path To Blocked
        --> Jump
    --> Go Left



In [ ]:
# Play the game for required amount games using the pre-programmed agent
# If you get a error of Cannot create multiple emulator instances then click on Kernel from drop down click Restart and just
# rerun all code blocks again
preprogrammedAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,True,1,behaviour_tree,3,blackboard)

------------------------------------
Game Num 1


C:\Users\Tiger\.conda\envs\game\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


## Convert game play to mp4
Update the below cmd to path where your game play is saved 

In [ ]:
%run -m retro.scripts.playback_movie RecordAgentsGamePlay/final_results_pre_programmed/AlienSoldier-Genesis-DefaultSettings.Level1-000000.bk2

# End